In [ ]:
%load_ext autoreload

%autoreload 2

import numpy as np
from lucifex.fdm import AB2, CN
from lucifex.sim import run, xdmf_to_npz
from lucifex.utils import grid, spacetime_grid, cross_section, maximum
from lucifex.viz import plot_colormap, create_animation, plot_line
from lucifex.io import write

from models.dns import dns_system_a


STORE = 1
WRITE = None
DIR_BASE = './data_test'
DIR_PARAMS = ('Nx', 'Ny')

sr = 0.2
simulation = dns_system_a(
    store_delta=STORE, 
    write_delta=WRITE, 
    dir_base=DIR_BASE, 
    dir_params=DIR_PARAMS,
    dir_timestamp=True,
)(
    Nx=150,
    Ny=150,
    Ra=2e3,
    Da=100.0,
    epsilon=1e-2,
    sr=sr,
    cr=0.0,
    s_limits=(0, sr),
    c_seed=(567, 765),
    c_limits=(0, 1),
    c_stabilization=None,
    secondary=True,
)

n_stop = 20
dt_init = 1e-6
n_init = 10
run(simulation, n_stop=n_stop, dt_init=dt_init, n_init=n_init)

if WRITE:
    xdmf_to_npz(simulation, delete_xdmf=False)

c, s, u, psi = simulation['c', 's', 'u', 'psi']
cCorr, sCorr = simulation['cCorr', 'sCorr']
mC, mD = simulation['mC', 'mD']
f, fBoundary = simulation['f', 'fBoundary']
fInterface, fPlus, fMinus = f.sub(0), f.sub(1), f.sub(2)
sMinMax, cMinMax, uMinMax = simulation['sMinMax', 'cMinMax', 'uMinMax']
cMin, cMax = cMinMax.sub(0), cMinMax.sub(1) 
uMax = uMinMax.sub(1)
dt, dtCFL, dtK = simulation['dt', 'dtCFL', 'dtK']


In [ ]:
c = simulation['c']
s = simulation['s']
plot_colormap(s.series[-1])

In [ ]:
plot_line(
    (c.time_series, [np.sum(i) for i in c.dofs_series])
)

In [ ]:
cCorr = simulation['cCorr']
plot_colormap(cCorr.series[500])
plot_line(
    (cCorr.time_series, [np.min(i) for i in cCorr.dofs_series]),
)

In [ ]:
plot_line(
    (uMax.time_series, uMax.value_series)
)
plot_line(
   [(cCorr.time_series, [np.min(i) for i in cCorr.dofs_series]),
     (cCorr.time_series, [np.max(i) for i in cCorr.dofs_series])]
)

In [ ]:
plot_line(
    (mC.time_series, mC.value_series)
)
plot_line(
    (mD.time_series, mD.value_series)
)
plot_line(
    (mC.time_series, [i + j for i, j in zip(mC.value_series, mD.value_series)])
)

In [ ]:
plot_line([
    (dt.time_series, dt.value_series),
    (dtCFL.time_series, dtCFL.value_series),
    (dtK.time_series, dtK.value_series),
    ],
    y_lims=(0, 0.1),
)

In [ ]:
plot_line((sMinMax.time_series, [i[1] for i in sMinMax.value_series]))

In [ ]:
plot_line((cMinMax.time_series, [i[0] for i in cMinMax.value_series]))

In [ ]:
from models.dns import interfacial_flux


interfacial_flux(c, ..., ..., 0.9, ...)

In [ ]:
for flux in f.split():
    flux_net = [f[0] - f[1] for f in flux.value_series]
    plot_line(
        [(flux.time_series, flux_net), (flux.time_series, flux.value_series)], 
        title=f'{flux.name}', 
        cyc='black',
        legend_labels=['net', 'advective', 'diffusive'],
    )

In [ ]:
for i in (0, 1, -1):
    fig, ax = plot_colormap(c.series[i])